In [1]:

import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from scipy.stats import lognorm
import os

cats = {
    "EDIP-Gold": "Depletion of Gold [kg]",
    "EDIP-Copper": "Copper [kg]",
    "ADP-Minerals- Total": "Depletion of Minerals [kg Sb-eq]",
    "EI-Minerals- Total": "Economic Importance [kg Sb-eq]",
    "EI-Minerals-Precious Metals": "Precious Metals",
    "GPR-Minerals- Total": "Geo-Politic Supply Risk [1]",
    "CML-climate change (GWP 100a)": "Green House Emissions",
    "CED - total": "Primary Energy [MJ-eq]",
    "EcoIndicator 99 (E - E) - total":"Total Impacts",
    "ReCiPe Endpoint (E - A) - total (total)": "Total Impacts",
}


sns.set_theme()
sns.set(context="paper", style="whitegrid", font="Times New Roman")
cm = 1/2.54
FONTSIZE = 8

def get_label(shape,scale):
    sigma = "\u03C3"
    mu ="\u03BC"
    geomUnc= np.exp(shape)
    scale_val = "{:.2E}".format(scale)
    geomUnc = "{:.4}".format(geomUnc)
    label = f"{mu} = {scale_val}, " + f"{sigma} = {geomUnc}"
    return label

In [2]:
ps_name = "3208-Mainboard"
cat_list = {"EDIP-Gold": "Depletion of Gold [kg]",
            "EI-Minerals- Total": "Economic Importance [kg Sb-eq]",
            "GPR-Minerals- Total": "Geo-Politic Supply Risk [1]",
            "CED - total": "Primary Energy [MJ-eq]",
            }
ps_names = [
"3204-CPU, mounted mainboard",
"3208-Mainboard",
"3211-PCB mounted mainboard",
"3219-PCB, for power supply unit",
"4440-Server, 1U, no storage",
"4443-Server, 2U, storage",
"4455-Blade (Ref. Unit)",
"5101-KDO-IT",
"6001-KDO"
]
for ps_name in ps_names:
    corr = "data/MC_TEMPRO_DB220425_{}_{}.csv".format("Corr", ps_name)
    df_mc_corr = pd.read_csv(corr)
    fig, axs = plt.subplots(2, 2, figsize=(22*cm, 20*cm))
    ax_ = [axs[i][j] for i in range(2) for j in range(2)]
    i=0
    for cat in cat_list.keys():
        data_corr = df_mc_corr[cat]
        data_corr = [x for x in data_corr if x > 0]
        for _ in range(10):
            shape, loc, scale = stats.lognorm.fit(data_corr, floc=0)
            data_corr = [x for x in data_corr if x < scale*np.exp(shape)**4]
            data_corr = [x for x in data_corr if x < scale*3]

        shape, loc, scale = stats.lognorm.fit(data_corr, floc=0)
        
        max_val = max(data_corr)
        min_val = min(data_corr)
        label_title = get_label(shape, scale)
        ax = ax_[i]
        ax.hist(data_corr, bins=40, density=True, alpha=0.6, color="b")
        ax.set_xlabel(cat_list[cat], fontsize=8)
        ax.set_ylabel('Probability Density', fontsize=8)
        ax.set_title(label_title,
                     fontsize=FONTSIZE, fontweight="bold")
        ax.grid(True)

        # Overlay the probability density function (pdf) of the lognormal distribution
        x = np.linspace(0, max_val*1.1, 10000)
        pdf = lognorm.pdf(x, shape, scale=scale)
        ax.plot(x, pdf, 'r-', lw=2, label='Lognormal PDF')
        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(FONTSIZE*0.75)
        sigma = np.exp(shape)
        ax.axvline(x=scale/sigma/sigma, color='grey',
                   label='95th, Upper', linestyle="--")
        ax.axvline(x=scale*sigma*sigma, color='grey',
                   label='95th, Lower', linestyle="--")
        ax.axvline(x=scale/sigma, color='green',
                   label='63th, Upper', linestyle="--")
        ax.axvline(x=scale*sigma, color='green',
                   label='63th, Lower', linestyle="--")
        ax.set_xlim(min_val/1.1, max_val)
        ax.legend(fontsize=8)

        if i !=2:
            ax_[i].get_legend().remove()
        else:
            ax_[i].legend(bbox_to_anchor=(1.1, -0.25), loc="lower center", ncol=5, fontsize=FONTSIZE)
        
        i+=1
    fig.savefig(f"SimpleResults/{ps_name}.png", dpi=900, bbox_inches='tight')
    plt.close()



